<a href="https://colab.research.google.com/github/iam-fern/Generate_addition_1X/blob/main/GQtest_cluster_data(y_hat)_add_delta_(_1_2)_n20_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import os
import statistics as stats
import scipy.stats as ss
import scipy.stats as st
from scipy.stats import f_oneway
from scipy import stats
import scipy.stats as stats
import time
import statsmodels.api as sm
from google.colab import drive
from statsmodels.formula.api import ols
from google.colab import files

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
drive.mount('/content/drive')

In [ ]:
base_path = '/content/drive/My Drive/pattern_addition_1X/addition_delta_-1.2'

In [ ]:
# เรียกใช้ข้อมูลจากไฟล์ โดยผ่านฟังก์ชัน ที่ชื่อว่า read_csv
file_path = os.path.join(base_path, 'data_add_delta_-1.2_n20.csv')

In [ ]:
df = pd.read_csv(file_path)
df

In [ ]:
df = pd.read_csv(file_path, index_col=['rep', 'index'])
df

In [ ]:
# ลบข้อมูลที่มี missing values
table = df.dropna()
table

In [ ]:
table.reset_index(inplace=True)
table

In [ ]:
table.set_index(['rep', 'index'], inplace=True)

In [ ]:
table

In [11]:
def get_result(run_replicate):
    new_table = table.loc[run_replicate]

    # show index
    new_table.reset_index(inplace=True)

    new_table.drop(['index'], axis=1, inplace=True)


    # แบ่งข้อมูลออกเป็น 2 กลุ่มด้วย k-mean

    #Set number of clusters
    x = new_table[["y_hat"]]

    #Another important parameter is init, which defines the method initialization to use (default='k-means++').
    #k-means++ algorithm uses the seeding technique for centroid initialization which is better than the random initialization method.
    kmeans = KMeans(n_clusters=2, init='k-means++', random_state=0).fit(x)

    #Generate cluster predictions and store in y_kmeans
    y_kmeans = kmeans.predict(x)

    #kmean_clusters.astype(float)
    kmean_clusters = pd.DataFrame({"clusters" : y_kmeans})

    # เชื่อมข้อมูลทั้ง 2 DataFrame เข้าด้วยกันด้วยคำสั่ง pd.concat
    data_clusters = pd.concat([new_table, kmean_clusters], axis=1)
    #data_clusters.reset_index(inplace=True)

    # กลุ่มที่ 1
    data_cluster0 = data_clusters[data_clusters.clusters == 0]

    y = data_cluster0[['Y']]
    x = data_cluster0[['x1']]
    x = sm.add_constant(x)
    model1 = sm.OLS(y, x).fit()

    DF1 = model1.df_resid + 1
    SSE1 = model1.ssr
    MSE1 = SSE1/DF1

    # กลุ่มที่ 2
    data_cluster1 = data_clusters[data_clusters.clusters == 1]

    y = data_cluster1[['Y']]
    x = data_cluster1[['x1']]
    x = sm.add_constant(x)
    model2 = sm.OLS(y, x).fit()

    DF2 = model2.df_resid + 1
    SSE2 = model2.ssr
    MSE2 = SSE2/DF2

    # คำนวณค่าสถิติ
    GQ_test_clusters = (SSE2/DF2) / (SSE1/DF1)

    df2 = DF2
    df1 = DF1

    # คำนวณค่า p-value
    # pdf(x, dfn, dfd, loc=0, scale=1) : Probability density function
    two_tailed = 2.0*(1.0-(ss.f.cdf(GQ_test_clusters, df1, df2)))
    p_greater = ss.f.cdf(GQ_test_clusters, df1, df2)
    p_less = 1.0-ss.f.cdf(GQ_test_clusters, df1, df2)

    return df1, df2, SSE1, SSE2, MSE1, MSE2, GQ_test_clusters, two_tailed, p_greater, p_less

# สร้าง list ของ replicates
list_replicate=list({i[0] for i in table.index})
list_replicate.sort()

# สร้าง list เพื่อเก็บค่า ทุกรอบที่ run
list_df1=[]
list_df2=[]
list_SSE1=[]
list_SSE2=[]
list_MSE1=[]
list_MSE2=[]
list_GQ_test_clusters=[]
list_result_test=[]
list_two_tailed=[]
list_p_greater=[]
list_p_less=[]
#list_result_test01=[]

# ใช้ for loop สำหรับการ run ทุก replicates
for run_replicate in list_replicate:
    df1, df2, SSE1, SSE2, MSE1, MSE2, GQ_test_clusters, two_tailed, p_greater, p_less=get_result(run_replicate)
    list_df1.append(df1)
    list_SSE1.append(SSE1)
    list_df2.append(df2)
    list_SSE2.append(SSE2)
    list_MSE1.append(MSE1)
    list_MSE2.append(MSE2)
    list_GQ_test_clusters.append(GQ_test_clusters)
    list_two_tailed.append(two_tailed)
    list_p_greater.append(p_greater)
    list_p_less.append(p_less)

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future versi

In [12]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
result=pd.DataFrame({"Replicate":list_replicate, "df1":list_df1, "df2":list_df2, "SSE1":list_SSE1,
                     "SSE2":list_SSE2, "MSE1":list_MSE1, "MSE2":list_MSE2,
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater, "less":list_p_less})
result.set_index(['Replicate'], inplace=True)
result=result[(result["two_tailed"]<=1) & (result["greater"]<=1) & (result["less"]<=1)]
result["Result_test"]=result["two_tailed"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                 if p_value < alpha else 'Reject005=1 : Homoscedasticity')
#two_tailed_new_show=two_tailed_new.drop(columns="greater")

result.astype(str)

,df1,df2,SSE1,SSE2,MSE1,MSE2,GQ_test_clusters,two_tailed,greater,less,Result_test
Replicate,,,,,,,,,,,
0.0,8.0,10.0,0.026811459173600456,0.06044644730406675,0.003351432396700057,0.006044644730406675,1.803600375874642,0.3769759886744499,0.811512005662775,0.18848799433722496,Reject005=1 : Homoscedasticity
1.0,11.0,7.0,0.05401621234842032,0.07000003779615328,0.004910564758947302,0.010000005399450469,2.036426743223362,0.3547295566284403,0.8226352216857798,0.17736477831422015,Reject005=1 : Homoscedasticity
3.0,9.0,9.0,0.021608214479109725,0.14732397160406935,0.0024009127199010804,0.01636933017822993,6.817961370500948,0.008629667009285358,0.9956851664953573,0.004314833504642679,Reject005=0 : Heteroscedasticity
4.0,5.0,13.0,0.03178116411552855,0.3738728062848915,0.00635623282310571,0.028759446637299346,4.524605601726091,0.026188314876780305,0.9869058425616098,0.013094157438390153,Reject005=0 : Heteroscedasticity
5.0,8.0,10.0,0.01844548917321174,0.05322465448781437,0.0023056861466514674,0.005322465448781437,2.308408478105842,0.2151501790463506,0.8924249104768247,0.1075750895231753,Reject005=1 : Homoscedasticity
6.0,11.0,7.0,0.03906464281226558,0.06673241482632382,0.003551331164751416,0.00953320211804626,2.6844024608765427,0.1999209412357683,0.9000395293821158,0.09996047061788416,Reject005=1 : Homoscedasticity
7.0,12.0,6.0,0.03444010627217985,0.30882749150861755,0.002870008856014988,0.05147124858476959,17.93417761652398,0.002017219448091856,0.9989913902759541,0.001008609724045928,Reject005=0 : Heteroscedasticity
8.0,13.0,5.0,0.07018643146643969,0.14051685772211556,0.005398956266649207,0.028103371544423113,5.205334171351811,0.07956441308416107,0.9602177934579195,0.039782206542080534,Reject005=1 : Homoscedasticity
11.0,13.0,5.0,0.08112173391894657,0.11838315122259353,0.006240133378380506,0.023676630244518708,3.794250668831613,0.1498467354101476,0.9250766322949262,0.0749233677050738,Reject005=1 : Homoscedasticity


In [13]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
result=pd.DataFrame({"Replicate":list_replicate, "df1":list_df1, "df2":list_df2, "SSE1":list_SSE1,
                     "SSE2":list_SSE2, "MSE1":list_MSE1, "MSE2":list_MSE2,
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater, "less":list_p_less})
result.set_index(['Replicate'], inplace=True)

result['two_tailed_new']=result[["two_tailed","greater","less"]].apply(lambda value: value[0] if value[0]<0.5 else (2*value[1] if value[1]<0.5 else 2*value[2]) , axis=1)
result["Result_test"]=result["two_tailed_new"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                     if p_value < alpha else 'Reject005=1 : Homoscedasticity')
#two_tailed_new_show=two_tailed_new.drop(columns="greater")
result_show=result.drop(columns=["two_tailed", "greater", "less"])
result_show.astype(str)

,df1,df2,SSE1,SSE2,MSE1,MSE2,GQ_test_clusters,two_tailed_new,Result_test
Replicate,,,,,,,,,
0.0,8.0,10.0,0.026811459173600456,0.06044644730406675,0.003351432396700057,0.006044644730406675,1.803600375874642,0.3769759886744499,Reject005=1 : Homoscedasticity
1.0,11.0,7.0,0.05401621234842032,0.07000003779615328,0.004910564758947302,0.010000005399450469,2.036426743223362,0.3547295566284403,Reject005=1 : Homoscedasticity
2.0,5.0,13.0,0.09136089787231462,0.05669104146811036,0.018272179574462925,0.004360849343700797,0.23866060017248794,0.12344361447180481,Reject005=1 : Homoscedasticity
3.0,9.0,9.0,0.021608214479109725,0.14732397160406935,0.0024009127199010804,0.01636933017822993,6.817961370500948,0.008629667009285358,Reject005=0 : Heteroscedasticity
4.0,5.0,13.0,0.03178116411552855,0.3738728062848915,0.00635623282310571,0.028759446637299346,4.524605601726091,0.026188314876780305,Reject005=0 : Heteroscedasticity
5.0,8.0,10.0,0.01844548917321174,0.05322465448781437,0.0023056861466514674,0.005322465448781437,2.308408478105842,0.2151501790463506,Reject005=1 : Homoscedasticity
6.0,11.0,7.0,0.03906464281226558,0.06673241482632382,0.003551331164751416,0.00953320211804626,2.6844024608765427,0.1999209412357683,Reject005=1 : Homoscedasticity
7.0,12.0,6.0,0.03444010627217985,0.30882749150861755,0.002870008856014988,0.05147124858476959,17.93417761652398,0.002017219448091856,Reject005=0 : Heteroscedasticity
8.0,13.0,5.0,0.07018643146643969,0.14051685772211556,0.005398956266649207,0.028103371544423113,5.205334171351811,0.07956441308416107,Reject005=1 : Homoscedasticity


In [14]:
result_show['Result_test'].value_counts()

Reject005=1 : Homoscedasticity      748
Reject005=0 : Heteroscedasticity    252
Name: Result_test, dtype: int64